In [1]:
import pandas as pd
import numpy as np
import copy

import torch
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score


In [2]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(14, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x
    

In [64]:
name_of_model='WW_WM_BW.pth'
model = DeepNet()
# model = Deep_wide_Net()
model.load_state_dict(torch.load(name_of_model))
model.eval()

DeepNet(
  (layer1): Linear(in_features=14, out_features=512, bias=True)
  (act1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (act2): ReLU()
  (layer3): Linear(in_features=256, out_features=60, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=60, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [65]:
print(model.state_dict().keys())


odict_keys(['layer1.weight', 'layer1.bias', 'layer2.weight', 'layer2.bias', 'layer3.weight', 'layer3.bias', 'output.weight', 'output.bias'])


Top-2 layers:
layer1.weight
layer2.weight


In [71]:
state_dict = model.state_dict()

# Convert the state dictionary to a tensor
tensor = torch.cat([value.flatten() for value in state_dict.values()])

# Apply the topk function to the tensor
_, top_i = torch.topk(tensor, k=2)

top_i

tensor([152, 150])

In [ ]:
layer_names = ['layer1.weight', 'layer1.bias', 'layer2.weight', 'layer2.bias', 'layer3.weight', 'layer3.bias']


In [76]:
weights = []
k=2
for name, param in model.named_parameters():
    if param.requires_grad:
        weights.append(torch.norm(param.data))

# Convert the weights to a tensor
weights_tensor = torch.stack(weights)

# Find the indices of the top-k layers
topk_indices = torch.topk(weights_tensor, k).indices

topk_indices

tensor([0, 2])

In [77]:
# Get the top-k layer names
layer_names = list(model.state_dict().keys())
topk_layers = [layer_names[idx] for idx in topk_indices]

print("Top-{} layers:".format(k))
for layer in topk_layers:
    print(layer)

Top-2 layers:
layer1.weight
layer2.weight


In [70]:
top_indices

tensor([18, 47])

In [40]:
alpha = {}
for name, _ in model.named_parameters():
    alpha[name] = model.state_dict()[name]

print(alpha)


{'layer1.weight': tensor([[-0.1236,  0.0936, -0.0858,  ..., -0.2022,  0.2071,  0.2143],
        [-0.0173, -0.0282,  0.0289,  ..., -0.0473,  0.0984, -0.1676],
        [ 0.2257, -0.0810, -0.0682,  ..., -0.0027, -0.1340, -0.0711],
        ...,
        [-0.0892, -0.2357,  0.0203,  ...,  0.2679, -0.0388, -0.2460],
        [ 0.2170,  0.2365,  0.2169,  ..., -0.1546,  0.0313,  0.2786],
        [-0.0207, -0.1458, -0.1536,  ...,  0.0709, -0.0735, -0.1318]]), 'layer1.bias': tensor([-0.0322,  0.0426, -0.0603,  0.0798,  0.1138,  0.1408,  0.0317,  0.2289,
         0.1325, -0.1165, -0.1265,  0.2285, -0.0939,  0.1958,  0.2569,  0.1675,
        -0.2616,  0.1116,  0.1401, -0.0685, -0.1891, -0.1634, -0.1235,  0.0884,
        -0.2367,  0.2376, -0.0763, -0.0003,  0.1408, -0.0229,  0.1229, -0.1721,
         0.0450, -0.2093,  0.0882, -0.2293, -0.2588,  0.1078, -0.0765,  0.0842,
        -0.0346, -0.0134, -0.2554,  0.0831, -0.1555,  0.0582,  0.1946, -0.2204,
         0.2441,  0.1677, -0.0461, -0.1893,  0.2440,

In [45]:
alpha['layer1.weight']

tensor([[-0.1236,  0.0936, -0.0858,  ..., -0.2022,  0.2071,  0.2143],
        [-0.0173, -0.0282,  0.0289,  ..., -0.0473,  0.0984, -0.1676],
        [ 0.2257, -0.0810, -0.0682,  ..., -0.0027, -0.1340, -0.0711],
        ...,
        [-0.0892, -0.2357,  0.0203,  ...,  0.2679, -0.0388, -0.2460],
        [ 0.2170,  0.2365,  0.2169,  ..., -0.1546,  0.0313,  0.2786],
        [-0.0207, -0.1458, -0.1536,  ...,  0.0709, -0.0735, -0.1318]])

In [9]:
all_params_name = [name for name in model.state_dict().keys()]

In [10]:
all_params_name

['layer1.weight',
 'layer1.bias',
 'layer2.weight',
 'layer2.bias',
 'layer3.weight',
 'layer3.bias',
 'output.weight',
 'output.bias']

In [11]:
trainable_params_name = [name for name, param in model.state_dict(keep_vars=True).items()
            if param.requires_grad ]

In [12]:
trainable_params_name

['layer1.weight',
 'layer1.bias',
 'layer2.weight',
 'layer2.bias',
 'layer3.weight',
 'layer3.bias',
 'output.weight',
 'output.bias']

In [13]:
a=model.state_dict()['layer1.weight']

In [14]:
a.shape

torch.Size([512, 14])

In [15]:
model.state_dict()['layer1.weight']

tensor([[-0.1236,  0.0936, -0.0858,  ..., -0.2022,  0.2071,  0.2143],
        [-0.0173, -0.0282,  0.0289,  ..., -0.0473,  0.0984, -0.1676],
        [ 0.2257, -0.0810, -0.0682,  ..., -0.0027, -0.1340, -0.0711],
        ...,
        [-0.0892, -0.2357,  0.0203,  ...,  0.2679, -0.0388, -0.2460],
        [ 0.2170,  0.2365,  0.2169,  ..., -0.1546,  0.0313,  0.2786],
        [-0.0207, -0.1458, -0.1536,  ...,  0.0709, -0.0735, -0.1318]])

In [16]:
model.state_dict().values()

odict_values([tensor([[-0.1236,  0.0936, -0.0858,  ..., -0.2022,  0.2071,  0.2143],
        [-0.0173, -0.0282,  0.0289,  ..., -0.0473,  0.0984, -0.1676],
        [ 0.2257, -0.0810, -0.0682,  ..., -0.0027, -0.1340, -0.0711],
        ...,
        [-0.0892, -0.2357,  0.0203,  ...,  0.2679, -0.0388, -0.2460],
        [ 0.2170,  0.2365,  0.2169,  ..., -0.1546,  0.0313,  0.2786],
        [-0.0207, -0.1458, -0.1536,  ...,  0.0709, -0.0735, -0.1318]]), tensor([-0.0322,  0.0426, -0.0603,  0.0798,  0.1138,  0.1408,  0.0317,  0.2289,
         0.1325, -0.1165, -0.1265,  0.2285, -0.0939,  0.1958,  0.2569,  0.1675,
        -0.2616,  0.1116,  0.1401, -0.0685, -0.1891, -0.1634, -0.1235,  0.0884,
        -0.2367,  0.2376, -0.0763, -0.0003,  0.1408, -0.0229,  0.1229, -0.1721,
         0.0450, -0.2093,  0.0882, -0.2293, -0.2588,  0.1078, -0.0765,  0.0842,
        -0.0346, -0.0134, -0.2554,  0.0831, -0.1555,  0.0582,  0.1946, -0.2204,
         0.2441,  0.1677, -0.0461, -0.1893,  0.2440,  0.1668, -0.0349, 

In [17]:
torch.stack(( model.state_dict()['layer1.weight'],), dim=-1)

tensor([[[-0.1236],
         [ 0.0936],
         [-0.0858],
         ...,
         [-0.2022],
         [ 0.2071],
         [ 0.2143]],

        [[-0.0173],
         [-0.0282],
         [ 0.0289],
         ...,
         [-0.0473],
         [ 0.0984],
         [-0.1676]],

        [[ 0.2257],
         [-0.0810],
         [-0.0682],
         ...,
         [-0.0027],
         [-0.1340],
         [-0.0711]],

        ...,

        [[-0.0892],
         [-0.2357],
         [ 0.0203],
         ...,
         [ 0.2679],
         [-0.0388],
         [-0.2460]],

        [[ 0.2170],
         [ 0.2365],
         [ 0.2169],
         ...,
         [-0.1546],
         [ 0.0313],
         [ 0.2786]],

        [[-0.0207],
         [-0.1458],
         [-0.1536],
         ...,
         [ 0.0709],
         [-0.0735],
         [-0.1318]]])

In [18]:
(a/a.sum()).shape

torch.Size([512, 14])

In [19]:
torch.sum(a/(a.sum()*torch.stack(( model.state_dict()['layer1.weight'],))))

tensor(108.5115)

In [22]:
# MLP

list(filter(lambda p: p.requires_grad,model.parameters()))

[Parameter containing:
 tensor([[-0.1236,  0.0936, -0.0858,  ..., -0.2022,  0.2071,  0.2143],
         [-0.0173, -0.0282,  0.0289,  ..., -0.0473,  0.0984, -0.1676],
         [ 0.2257, -0.0810, -0.0682,  ..., -0.0027, -0.1340, -0.0711],
         ...,
         [-0.0892, -0.2357,  0.0203,  ...,  0.2679, -0.0388, -0.2460],
         [ 0.2170,  0.2365,  0.2169,  ..., -0.1546,  0.0313,  0.2786],
         [-0.0207, -0.1458, -0.1536,  ...,  0.0709, -0.0735, -0.1318]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0322,  0.0426, -0.0603,  0.0798,  0.1138,  0.1408,  0.0317,  0.2289,
          0.1325, -0.1165, -0.1265,  0.2285, -0.0939,  0.1958,  0.2569,  0.1675,
         -0.2616,  0.1116,  0.1401, -0.0685, -0.1891, -0.1634, -0.1235,  0.0884,
         -0.2367,  0.2376, -0.0763, -0.0003,  0.1408, -0.0229,  0.1229, -0.1721,
          0.0450, -0.2093,  0.0882, -0.2293, -0.2588,  0.1078, -0.0765,  0.0842,
         -0.0346, -0.0134, -0.2554,  0.0831, -0.1555,  0.0582,  0.1946, -0.2204

In [21]:
model.parameters()

<generator object Module.parameters at 0x0000026F56F60E40>

In [20]:
torch.sum(a / a.sum() * torch.stack((model.state_dict()['layer1.weight'],), dim=-1), dim=-1

RuntimeError: The size of tensor a (512) must match the size of tensor b (14) at non-singleton dimension 1

In [64]:
print("Shape of a:", a.shape)
print("Shape of layer1.weight:", model.state_dict()['layer1.weight'].shape)


Shape of a: torch.Size([512, 14])
Shape of layer1.weight: torch.Size([512, 14])
